In [1]:
import numpy as np
import pandas as pd
import calendar
import copy
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,MinMaxScaler
import calendar
import datetime
from datetime import datetime
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import preprocessingWithMissingvalues

In [2]:
carDf= pd.read_csv('oneHotPreprocess.csv')
carDf.head()

,Make_Accura,Make_BMW,Make_Chevrolet,Make_Dodge,Make_Ferrari,Make_Ford,Make_Honda,Make_Jaguar,Make_Lexus,Make_Mazda,...,NumberOfCars_5 to 8,NumberOfCars_more than 8,BasePolicy_All Perils,BasePolicy_Collision,BasePolicy_Liability,Age,Deductible,DriverRating,DaysDiff,Lable
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0.2625,0.000000,0.000000,0.071795,0
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0.4250,1.000000,1.000000,0.069231,0
2,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0.5875,0.666667,0.666667,0.089744,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0.8125,0.333333,0.333333,0.107692,0
4,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0.3375,0.000000,0.000000,0.094872,0


In [3]:
#get the lable from the dataset
carDf.head()
carLable= carDf['Lable']
carDf.drop(['Lable'],inplace=True,axis=1) #drop the lable;

In [4]:
carDateNormalized= carDf

In [5]:
#Divide the data into fraudulent and non-fraudulent data
columnlable= list(carDateNormalized.columns.values)

#create the dataframe for fraudulent and non-fraudulent data
nonFraudulent= pd.DataFrame(columns=columnlable)
nonFraudulentLable= pd.DataFrame(columns=['lable'])

fraudulent= pd.DataFrame(columns=columnlable)
fraudulentLable= pd.DataFrame(columns=['lable'])

print("nonFraudulent:",type(nonFraudulent))
print("carDateNormalized:",type(carDateNormalized))

j= 0
k= 0
for i in range(carDateNormalized.shape[0]):
    if(carLable[i]==0):
        nonFraudulent.loc[j]= carDateNormalized.loc[i]
        nonFraudulentLable.loc[j]= 0.0
        j += 1
    else:
        fraudulent.loc[k]= carDateNormalized.loc[i]
        fraudulentLable.loc[i]= 1.0
        k += 1


nonFraudulent: <class 'pandas.core.frame.DataFrame'>
carDateNormalized: <class 'pandas.core.frame.DataFrame'>


In [6]:
print(type(nonFraudulent))
print(type(fraudulent))

#convert to dataframe
nonFraudulentLable= pd.DataFrame(nonFraudulentLable)
fraudulentLable= pd.DataFrame(fraudulentLable)


<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [7]:
#split the dataset into train and test set in equal proportion
#divide the nonFraudulent into test and train
nonFraudX_train,nonFraudX_test,nonFraudY_train,nonFraudY_test = train_test_split(nonFraudulent,nonFraudulentLable,random_state=3,test_size=0.3)

print('nonFraudxtrain:',type(nonFraudX_train))
print(nonFraudX_test.shape)

#divide the fraudulent into test and train
fraudX_train,fraudX_test,fraudY_train,fraudY_test = train_test_split(fraudulent, fraudulentLable,random_state=3,test_size=0.3)

print('fraudxtrain:',type(fraudX_train))
print(fraudX_test.shape)

nonFraudxtrain: <class 'pandas.core.frame.DataFrame'>
(4349, 85)
fraudxtrain: <class 'pandas.core.frame.DataFrame'>
(277, 85)


In [8]:
#combine the fraudulent and nonfraudulent training and test set to make the newTrain and newTest

#create the dataframe for fraudulent and non-fraudulent data
newCombineTrain= pd.DataFrame(columns=columnlable)
newCombineTrainLable= pd.DataFrame(columns=['lable'])

newCombineTest= pd.DataFrame(columns=columnlable)
newCombineTestLable= pd.DataFrame(columns=['lable'])



#combine trainset
framesFeatures= [nonFraudX_train,fraudX_train]
framesLable= [nonFraudY_train,fraudY_train]
newCombineTrain= pd.concat(framesFeatures)
newCombineTrainLable= pd.concat(framesLable)


framesFeatures2= [nonFraudX_test,fraudX_test]
framesLable2= [nonFraudY_test,fraudY_test]
newCombineTest= pd.concat(framesFeatures2)
newCombineTestLable= pd.concat(framesLable2)

#make the dataset as values
newCombineTrain= newCombineTrain.values
newCombineTrainLable= newCombineTrainLable.values

newCombineTest= newCombineTest.values
newCombineTestLable= newCombineTestLable.values

print("TrainData set size:",newCombineTrain.shape[0])
print("Test Data set size:",newCombineTest.shape[0])

TrainData set size: 10793
Test Data set size: 4626


In [9]:
#create the class Dataset which returns the data and labels
from torch.utils.data.dataset import Dataset, TensorDataset
class myDataset(Dataset):
    
    def __init__(self,features,lables,transform= None):
        dataTensor= []
        lableTensor= []
        dataSize= features.shape[0]
        
        for data in range(dataSize):
            feature= features[data,:]
            #feature= torch.from_numpy(feature).float()
            feature= torch.Tensor(feature)
            dataTensor.append(feature)
            
            lable= np.asanyarray(lables[data])
            lable= torch.from_numpy(lable).float()
           
            #lable= torch.Tensor(lable)
            #print("lable:",lable)
            #assert(False)
           
            lableTensor.append(lable)
        
        #put everything in features and lables
        self.features= dataTensor
        self.lables= lableTensor
    
    
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self,index):
        
        feature= self.features[index]
        lable= self.lables[index]
        #print("get_item feature:",feature)
        #print("get_item lable:",lable)
        return feature,lable

In [10]:
#make the for train and test loader
myTrainDataset= myDataset(newCombineTrain,newCombineTrainLable)
myTestDataset= myDataset(newCombineTest,newCombineTestLable)

In [11]:
#make the trainloader and test loader for nonfraudulent dataset.
trainLoader= torch.utils.data.DataLoader(myTrainDataset,batch_size=8,shuffle=True,num_workers=0)
testLoader= torch.utils.data.DataLoader(myTestDataset,batch_size=1,shuffle=True,num_workers=0)

newTrainLoader= torch.utils.data.DataLoader(myTrainDataset,batch_size=1,shuffle=True,num_workers=0)



In [12]:
#Network architecture for the base autoencoders
class Network(nn.Module):
    def __init__(self):
        super(Network,self).__init__()
        self.fc1= nn.Linear(85,64)
        self.act1= nn.LeakyReLU()
        self.fc2= nn.Linear(64,43)
        self.act2= nn.LeakyReLU()
        self.fc3= nn.Linear(43,24)
        self.act3= nn.LeakyReLU()
        self.fc4= nn.Linear(24,10)
        self.act4= nn.LeakyReLU()
        self.fc5= nn.Linear(10,2)
        self.act5= nn.Sigmoid();
            
    def forward(self,x):
        x= self.fc1(x)
        x= self.act1(x)
        x= self.fc2(x)
        x= self.act2(x)
        x= self.fc3(x)
        x= self.act3(x)
        x= self.fc4(x)
        x= self.act4(x)
        x= self.fc5(x)
        x= self.act5(x)
        return x;

In [13]:
model= Network()
#criterion= nn.CrossEntropyLoss()
criterion= nn.BCELoss()
optimizer= torch.optim.SGD(model.parameters(), lr=0.0001,momentum=0.9,weight_decay=0.0001)

In [14]:
#train the model
training_loss= []
for epoch in range(500):
    validation_loss= []
    
    running_loss= 0.0
    for i, (feature,lable) in enumerate(trainLoader):
        
        #gets the inputs
        inputs= torch.tensor(feature)
        lables= torch.tensor(lable).float()
        #lables= lable.type(torch.LongTensor)
        #print("lables:",lables)
        #print("lables.size:",lables.size())
       
        # =====================forward====================
        output = model(inputs)
        
        #print("output:",output)
        #print("output.size:",output.size())
        
        #print("output.requires_grad:",output.requires_grad)
        
        """
        maxIndex= torch.argmax(output,dim=1)
        maxProb= []
        tensorLength= output.size(0)
        print("maxIndex:",maxIndex)
        
        for i in range(tensorLength):
            index= maxIndex[i]
            maxProb.append(output[i][index].item())
        newMaxProb= torch.tensor(maxProb)
        newMaxProb= newMaxProb.view(-1,1)
        """
        
        newMaxProb,_ = torch.topk(output,1,dim=1)
        #print("newMaxProb:",newMaxProb)
        #print("newMaxProb.size:",newMaxProb.size())
        #print("newMaxProb.requires_grad:",newMaxProb.requires_grad)
        
        
        
        #loss= criterion(output,lables.squeeze())
        #output= newMaxProb
        loss= criterion(newMaxProb,lables.squeeze())
        
        #print("loss:",loss)
        
        # ===================backward====================
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # =======print the statistics
        running_loss += loss.item()
        
    print('[%d] loss: %.6f' %(epoch + 1,  running_loss /newCombineTrain.shape[0]))
    training_loss.append(running_loss/newCombineTrain)
    
    

/usr/local/anaconda35/lib/python3.6/site-packages/torch/nn/functional.py:1594: UserWarning: Using a target size (torch.Size([8])) that is different to the input size (torch.Size([8, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))
/usr/local/anaconda35/lib/python3.6/site-packages/torch/nn/functional.py:1594: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))
/usr/local/anaconda35/lib/python3.6/site-packages/ipykernel_launcher.py:59: RuntimeWarning: divide by zero encountered in true_divide


[1] loss: 0.071745
[2] loss: 0.055792
[3] loss: 0.044250
[4] loss: 0.035953
[5] loss: 0.031164
[6] loss: 0.029165
[7] loss: 0.028537
[8] loss: 0.028367
[9] loss: 0.028308
[10] loss: 0.028273
[11] loss: 0.028239
[12] loss: 0.028206
[13] loss: 0.028175
[14] loss: 0.028143
[15] loss: 0.028334
[16] loss: 0.028076
[17] loss: 0.028047
[18] loss: 0.028014
[19] loss: 0.027979
[20] loss: 0.027942
[21] loss: 0.028133
[22] loss: 0.027867
[23] loss: 0.027827
[24] loss: 0.027786
[25] loss: 0.027738
[26] loss: 0.027697
[27] loss: 0.027648
[28] loss: 0.027597
[29] loss: 0.027541
[30] loss: 0.027489
[31] loss: 0.027429
[32] loss: 0.027366
[33] loss: 0.027298
[34] loss: 0.027233
[35] loss: 0.027155
[36] loss: 0.027078
[37] loss: 0.026996
[38] loss: 0.026905
[39] loss: 0.026815
[40] loss: 0.026716
[41] loss: 0.026619
[42] loss: 0.026508
[43] loss: 0.026403
[44] loss: 0.026286
[45] loss: 0.026168
[46] loss: 0.026050
[47] loss: 0.025928
[48] loss: 0.025806
[49] loss: 0.025678
[50] loss: 0.025564
[51] loss

[397] loss: 0.018857
[398] loss: 0.018762
[399] loss: 0.018758
[400] loss: 0.018829
[401] loss: 0.018720
[402] loss: 0.018715
[403] loss: 0.018749
[404] loss: 0.018449
[405] loss: 0.018705
[406] loss: 0.018465
[407] loss: 0.018553
[408] loss: 0.018702
[409] loss: 0.018426
[410] loss: 0.018392
[411] loss: 0.018260
[412] loss: 0.018366
[413] loss: 0.018274
[414] loss: 0.018469
[415] loss: 0.018156
[416] loss: 0.018212
[417] loss: 0.018088
[418] loss: 0.018234
[419] loss: 0.018035
[420] loss: 0.018010
[421] loss: 0.018156
[422] loss: 0.018020
[423] loss: 0.018079
[424] loss: 0.017880
[425] loss: 0.017963
[426] loss: 0.017845
[427] loss: 0.017861
[428] loss: 0.017844
[429] loss: 0.017889
[430] loss: 0.017883
[431] loss: 0.017632
[432] loss: 0.017609
[433] loss: 0.017665
[434] loss: 0.017684
[435] loss: 0.017679
[436] loss: 0.017718
[437] loss: 0.017601
[438] loss: 0.017529
[439] loss: 0.017546
[440] loss: 0.017656
[441] loss: 0.017638
[442] loss: 0.017574
[443] loss: 0.017423
[444] loss: 0

In [15]:
#predict the model:
#function for testing the model
def testModel(loader):
    predictedLabel= []
    maxList= []
    trueLable= []
    maxProb= []
    for i,(feature,lable) in enumerate(loader):
        inputs= torch.tensor(feature)
        output= model(inputs)
        newMaxProb,maxIndex = torch.topk(output,1,dim=1)
        
        #combine= torch.sum(output)
        #print("combine:",combine)
        #assert(False)
        '''        print("maxIndex:",maxIndex)
        if(i<100):
            print("output:",output)
            print("newMaxProb:",newMaxProb)
            print("newMaxProb.size:",newMaxProb.size())
        else:
            assert(False)
        
        '''
        
        trueLable.append(lable)
        #predictedLabel.append(maxIndex.item())
        #maxList.append(maxProb)
        

        
        if(newMaxProb>0.095):
            predictedLabel.append(1)
        else:
            predictedLabel.append(0)
        
    
    return predictedLabel,trueLable,maxProb


In [16]:
#test the model on test set
predictedLable,trueLable,maxList= testModel(testLoader)



In [17]:
totalLength= len(trueLable)
TP=FP=FN=TN= 0
for i in range(totalLength):
    if(int(trueLable[i])==1 and predictedLable[i]==1):
        TP += 1
    elif(int(trueLable[i])==1 and predictedLable[i]==0):
        FN += 1
    elif(int(trueLable[i])==0 and predictedLable[i]==0):
        TN += 1
    elif(int(trueLable[i])==0 and predictedLable[i]==1):
        FP += 1

print("\n")
print("\nTP:",TP)
print("\nFN:",FN)
print("\nFP:",FP)
print("\nTN:",TN)

print("\nResult ")
print("Accuracy:",(TP+TN)/(TP+FP+FN+TN)*100)
print("Sensitivity:",TP/(TP+FN)*100)
print("Specificity:",TN/(TN+FP)*100)





TP: 123

FN: 154

FP: 651

TN: 3698

Result 
Accuracy: 82.59835711197579
Sensitivity: 44.4043321299639
Specificity: 85.03104161876294


In [18]:
#code for the ranking algorithm to test whether the model is correct or not.
def ranking(loader):
    
    probabilityDic= {}
    
    for i,(feature,lable) in enumerate(loader):
        inputs= torch.tensor(feature)
        output= model(inputs)
        #print(lable.item())
        #print("output:",output.item())
        probabilityDic[output.item()]= lable.item()
    return probabilityDic


In [19]:
probabilityDic= ranking(newTrainLoader)

print("Length of Dictionary:",len(probabilityDic))

ValueError: only one element tensors can be converted to Python scalars

In [ ]:
#sort the 
#sort the dictionary
sorted(probabilityDic.keys())



In [ ]:
#create the bucket
dictLength= len(probabilityDic)
print(dictLength)

In [ ]:
keyList= list(probabilityDic.keys())
valueList= list(probabilityDic.values())
print("length:",len(keyList))

In [ ]:
#create the 10 buckets of equal size except 1
bucket1=[]
bucket2=[]
bucket3=[]
bucket4=[]
bucket5=[]
bucket6=[]
bucket7=[]
bucket8=[]
bucket9=[]
bucket10=[]

In [ ]:
for i in range(len(keyList)):
    if(i<1159):
        bucket1.append(valueList[i])
    elif(i<2309):
        bucket2.append(valueList[i])
    elif(i<3459):
        bucket3.append(valueList[i])
    elif(i<4609):
        bucket4.append(valueList[i])
    elif(i<5759):
        bucket5.append(valueList[i])
    elif(i<6909):
        bucket6.append(valueList[i])
    elif(i<8059):
        bucket7.append(valueList[i])
    elif(i<9209):
        bucket8.append(valueList[i])
    elif(i<10359):
        bucket9.append(valueList[i])
    else:
        bucket10.append(valueList[i])

In [ ]:
bucket1_OneCount= bucket1.count(1)
print("bucket1_OneCount:",bucket1_OneCount)

bucket2_OneCount= bucket2.count(1)
print("bucket2_OneCount:",bucket2_OneCount)

bucket3_OneCount= bucket3.count(1)
print("bucket3_OneCount:",bucket3_OneCount)

bucket4_OneCount= bucket4.count(1)
print("bucket4_OneCount:",bucket4_OneCount)

bucket5_OneCount= bucket5.count(1)
print("bucket5_OneCount:",bucket5_OneCount)

bucket6_OneCount= bucket6.count(1)
print("bucket6_OneCount:",bucket6_OneCount)

bucket7_OneCount= bucket7.count(1)
print("bucket7_OneCount:",bucket7_OneCount)

bucket8_OneCount= bucket8.count(1)
print("bucket8_OneCount:",bucket8_OneCount)

bucket9_OneCount= bucket9.count(1)
print("bucket9_OneCount:",bucket9_OneCount)

bucket10_OneCount= bucket10.count(1)
print("bucket10_OneCount:",bucket10_OneCount)